# ELT - Fase 1: Limpieza de datos segun requerimientos minimos de Henry

En esta seccion, limpiamos el dataset movies_dataset.csv segun los requerimientos minimos de Henry; son 6 instrucciones en total:
1. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.
2. De haber fechas, deberán tener el formato AAAA-mm-dd
3. Los valores nulos del campo release date deben eliminarse
4. Deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
5. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0
6. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

In [1]:
# requisitos
import os
import pandas as pd
import numpy as np

data_movies = pd.read_csv(os.path.join('1_data','movies_dataset.zip'),compression='zip',low_memory=False).convert_dtypes()
data_movies.rename(columns = {'id':'pelicula_id'}, inplace = True) # definimos nuestro primary key

## Instruccion 1

Eliminar las columnas que no serán utilizadas: video, imdb_id, adult, original_title, poster_path y homepage.

In [2]:
columns_excluidas_henry=[ # Instruccion: Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage
    'video',
    'imdb_id',
    'adult',
    'original_title',
    'poster_path',
    'homepage']
columns_excluidas_crtrprsnl=['status','tagline'] # criterio personal: no relevantes para modelo ML
columns_desanidar=[
    'belongs_to_collection',
    'genres',
    'production_companies',
    'production_countries',
    'spoken_languages']
data_movies_subset=data_movies.drop(list(columns_excluidas_henry+columns_excluidas_crtrprsnl+columns_desanidar), axis=1)
data_movies_subset.info()
data_movies_subset.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45466 non-null  string 
 1   pelicula_id        45466 non-null  string 
 2   original_language  45455 non-null  string 
 3   overview           44512 non-null  string 
 4   popularity         45461 non-null  string 
 5   release_date       45379 non-null  string 
 6   revenue            45460 non-null  Int64  
 7   runtime            45203 non-null  Int64  
 8   title              45460 non-null  string 
 9   vote_average       45460 non-null  Float64
 10  vote_count         45460 non-null  Int64  
dtypes: Float64(1), Int64(3), string(7)
memory usage: 4.0 MB


,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Toy Story,7.7,5415
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Jumanji,6.9,2413


Esperamos que las siguientes columnas sean numericas: pelicula_id
pero no lo es.

In [4]:
#data_movies_subset['pelicula_id']=pd.to_numeric(data_movies_subset['pelicula_id']) # ValueError: Unable to parse string "1997-08-20" at position 19730
# inspeccionamos visualmente el error arojado en la posicion 19730 de pelicula_id
data_movies_subset.iloc[19730]['pelicula_id'] # algunas filas puede contener valores que no corresponden al valor esperado

'1997-08-20'

In [7]:
# identificamos las filas que contienen datos tipo fecha en la columna de nuestro primary key: pelicula_id
def check_type(x):
     try: 
        return type(eval(x)) 
     except Exception as e:
        return type(x)

data_movies_subset.loc[data_movies_subset['pelicula_id'].map(check_type)!=int]

,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count
19730,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,1997-08-20,104.0,Released,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>
29503,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,2012-09-29,68.0,Released,<NA>,12,<NA>,<NA>,<NA>,<NA>,<NA>
35587,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,2014-01-01,82.0,Released,Beware Of Frost Bites,22,<NA>,<NA>,<NA>,<NA>,<NA>


In [8]:
# removemos las filas que contienen fecha en la columna de primary key
data_movies_subset.drop(
    data_movies_subset.loc[data_movies_subset['pelicula_id'].map(check_type)!=int].index,
    inplace=True)
print("Numero de filas que no contienen datos tipo fecha en la columna ['pelicula_id']:",
    data_movies_subset.loc[data_movies_subset['pelicula_id'].map(check_type)!=int].shape[0]
)

Numero de filas que no contienen datos tipo fecha en la columna ['pelicula_id']: 0


## Instruccion 2
De haber fechas, deberán tener el formato AAAA-mm-dd

In [9]:
columns_numeric=[
    'pelicula_id',
    'budget',
    'revenue',
    'popularity',
    'vote_average',
    'vote_count',
    'runtime']

columns_date=['release_date']

data_movies_subset[columns_numeric]=data_movies_subset[columns_numeric].apply(pd.to_numeric) # convertimos a numerica
data_movies_subset[columns_date]=data_movies_subset[columns_date].apply(pd.to_datetime) # convertimos a fecha 

In [10]:
data_movies_subset_limpia=data_movies_subset.convert_dtypes() # volvemos a evaluar el df
data_movies_subset_limpia.info()
data_movies_subset_limpia.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45463 non-null  Int64         
 1   pelicula_id        45463 non-null  Int64         
 2   original_language  45452 non-null  string        
 3   overview           44509 non-null  string        
 4   popularity         45460 non-null  Float64       
 5   release_date       45376 non-null  datetime64[ns]
 6   revenue            45460 non-null  Int64         
 7   runtime            45203 non-null  Int64         
 8   title              45460 non-null  string        
 9   vote_average       45460 non-null  Float64       
 10  vote_count         45460 non-null  Int64         
dtypes: Float64(2), Int64(5), datetime64[ns](1), string(3)
memory usage: 4.5 MB


,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Toy Story,7.7,5415
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Jumanji,6.9,2413
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0,101,Grumpier Old Men,6.5,92


## Instruccion 3
Los valores nulos del campo release date deben eliminarse

In [42]:
# instruccion 1: Los valores nulos del campo release date deben eliminarse
data_movies_subset_limpia['release_date'].isna().any()
print("Cantidad de valores nulos en ['release_date']:",
    data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].shape[0])
data_movies_subset_limpia['release_date'].isna()
rows_release_date_na = data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].index
data_movies_subset_limpia.drop(rows_release_date_na, inplace=True)
print("Cantidad de valores nulos en ['release_date'] despues del drop:",
    data_movies_subset_limpia.loc[data_movies_subset_limpia['release_date'].isna()].shape[0])

Cantidad de valores nulos en ['release_date']: 87
Cantidad de valores nulos en ['release_date'] despues del drop: 0


## Instruccion 4
Deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [43]:
data_movies_subset_limpia['release_year'] = data_movies_subset_limpia['release_date'].dt.year

## Instruccion 5

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0

In [44]:
# Cantidad de valores nulos en ['budget','revenue']
data_movies_subset_limpia[['budget','revenue']].isna().sum()

budget     0
revenue    0
dtype: int64

In [45]:
data_movies_subset_limpia[['budget','revenue']] = data_movies_subset_limpia[['budget','revenue']].fillna(0)
data_movies_subset_limpia[['budget','revenue']].isna().sum()

budget     0
revenue    0
dtype: int64

## Instruccion 6
Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

In [46]:
data_movies_subset_limpia['return']=data_movies_subset_limpia['revenue']/data_movies_subset_limpia['budget'] # nueva columna return
data_movies_subset_limpia['return']=np.nan_to_num(data_movies_subset_limpia['return']) # convertimos a np.num para facilar el reemplazo a zeros
data_movies_subset_limpia['return'].fillna(0, inplace=True) # reemplazamos por zero cuando no hay datos para el caso de NaN
data_movies_subset_limpia['return'].replace([np.inf, -np.inf], 0, inplace=True) # reemplazamos por zero cuando no hay datos para el caso de inf
print("Hay valores nulos en cualquiera de ['revenue','budget','return']:",
    any(data_movies_subset_limpia[['revenue','budget','return']].isna().any()))


Hay valores nulos en cualquiera de ['revenue','budget','return']: False


In [47]:
# Removemos duplicados
data_movies_subset_limpia.drop_duplicates(inplace=True) # no funciona; deben existir filas que no son exactamente iguales.
print("Numero de peliculas unicas:",data_movies_subset_limpia['pelicula_id'].nunique(),"\n")
print("Numero de registros (filas):",data_movies_subset_limpia.shape[0],"\n")
print("Numero de peliculas con informacion inconsiste:",data_movies_subset_limpia.shape[0]-data_movies_subset_limpia['pelicula_id'].nunique(),"\n")

Numero de peliculas unicas: 45346 

Numero de registros (filas): 45359 

Numero de peliculas con informacion inconsiste: 13 



In [48]:
# para una misma pelicula existen valores distintos.
data_movies_subset_inconsistent=data_movies_subset_limpia.loc[data_movies_subset_limpia.duplicated(subset=['pelicula_id'], keep=False)]
print("Numero de filas con informacion duplicada en algunas columnas:",
    data_movies_subset_inconsistent.shape[0])
print("Numero de peliculas unicas con informacion duplicadas en algunas columnas:",
    data_movies_subset_inconsistent['pelicula_id'].nunique())
data_movies_subset_inconsistent.sort_values(['pelicula_id']).head(4)

Numero de filas con informacion duplicada en algunas columnas: 26
Numero de peliculas unicas con informacion duplicadas en algunas columnas: 13


,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return
33826,30000000,4912,en,"Television made him famous, but his biggest hi...",7.645827,2002-12-30,33013805,113,Confessions of a Dangerous Mind,6.6,281,2002,1.100460
5865,30000000,4912,en,"Television made him famous, but his biggest hi...",11.331072,2002-12-30,33013805,113,Confessions of a Dangerous Mind,6.6,281,2002,1.100460
4114,16000000,10991,ja,When Molly Hale's sadness of her father's disa...,10.264597,2000-07-08,68411275,93,Pokémon: Spell of the Unknown,6.0,143,2000,4.275705
44821,16000000,10991,ja,When Molly Hale's sadness of her father's disa...,6.480376,2000-07-08,68411275,93,Pokémon: Spell of the Unknown,6.0,144,2000,4.275705


In [49]:
# Solucionamos tomando la primera entrada para cada pelicula, y descartamos el resto.
data_movies_subset_limpia_sindups=data_movies_subset_limpia.loc[~data_movies_subset_limpia.duplicated(subset=['pelicula_id'], keep='first')]
print("Numero de peliculas unicas:",data_movies_subset_limpia_sindups['pelicula_id'].nunique(),"\n")
print("Numero de registros (filas):",data_movies_subset_limpia_sindups.shape[0],"\n")
print("Numero de peliculas con informacion inconsiste:",
      data_movies_subset_limpia_sindups.shape[0]-data_movies_subset_limpia_sindups['pelicula_id'].nunique(),"\n")


Numero de peliculas unicas: 45346 

Numero de registros (filas): 45346 

Numero de peliculas con informacion inconsiste: 0 



## Finalizamos
Guardamos el dataset limpio

In [50]:
# Nuestro dataset final
print("Universo de peliculas con datos unicos:",data_movies_subset_limpia_sindups['pelicula_id'].nunique(),"\n")
data_movies_subset_limpia_sindups.to_csv(os.path.join("2_pipeline","data_movies_subset_limpia.csv"))
data_movies_subset_limpia_sindups.info()
data_movies_subset_limpia_sindups.head(2)


Universo de peliculas con datos unicos: 45346 

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   budget             45346 non-null  Int64         
 1   pelicula_id        45346 non-null  Int64         
 2   original_language  45335 non-null  string        
 3   overview           44405 non-null  string        
 4   popularity         45346 non-null  Float64       
 5   release_date       45346 non-null  datetime64[ns]
 6   revenue            45346 non-null  Int64         
 7   runtime            45100 non-null  Int64         
 8   title              45346 non-null  string        
 9   vote_average       45346 non-null  Float64       
 10  vote_count         45346 non-null  Int64         
 11  release_year       45346 non-null  int32         
 12  return             45346 non-null  float64       
dtypes: Float64(2), Int

,budget,pelicula_id,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81,Toy Story,7.7,5415,1995,12.451801
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104,Jumanji,6.9,2413,1995,4.043035
